# OpenCV vs Kornia (CPU vs GPU) Computer Vision Benchmarks
This notebook investigates the performance improvements between the OpenCV and Kornia libraries. Kornia is a library that relies on GPU in order to accelerate differntiable computer vision tasks. In the backend, it uses Pytorch and represents computer vision tasks as computational graphs to accelerate them on GPUs.

##  Local Development Hardware:
**CPU:** Intel Core i9-12900K <br>
**GPU:** NVIDIA Geforce RTX 3090 <br>

<a target="_blank" href="https://colab.research.google.com/github/dotimothy/AccelerationMethods/blob/main/OpenCVvsKornia.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>  